In [0]:
!pip install -U tensorflow

In [0]:
from google.colab import files
import zipfile
from tensorflow import keras
import os
from shutil import copyfile
import numpy as np

from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img

In [0]:
chk = os.listdir('.')
print(chk)

In [0]:
!rm 'test_subjects4.zip'

In [0]:
uploaded = files.upload()

In [0]:
zpf = zipfile.ZipFile("test_subjects4.zip")
zpf.extractall()
zpf.close()

In [0]:
!mkdir a
!mkdir b
!mkdir c
!mkdir a/images
!mkdir b/images
!mkdir c/images

In [0]:
!rm -r dataset

In [0]:
print(os.listdir('test_subjects4'))

In [0]:
files = os.listdir('test_subjects4')
for f in files:
  cmn = './test_subjects4/'
  if f[-1] == 'g':
    if f[0] == 'a':
      copyfile(cmn + f, './a/images/' + f[1:])
    if f[0] == 'b':
      copyfile(cmn + f, './b/images/' + f[1:])
    if f[0] == 'c':
      copyfile(cmn + f, './c/images/' + f[1:])

In [0]:
print(os.listdir('./a/images'))

In [0]:
table_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)
col_mask_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)
cell_mask_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)

In [0]:
seed=1
table_generator = table_datagen.flow_from_directory('./a', target_size=(331, 701), class_mode=None, batch_size=10, seed=seed)
col_mask_generator = col_mask_datagen.flow_from_directory('./b', target_size=(331, 701), class_mode=None, batch_size=10, seed=seed)
cell_mask_generator = cell_mask_datagen.flow_from_directory('./c', target_size=(331, 701), class_mode=None, batch_size=10, seed=seed)

In [0]:
array_to_img(RGBtoBW(np.array([tab])))

In [0]:
def new_generator(a,b):
  while True:
    a1 = a.next()
    b1 = b.next()
    yield(a1, b1)

In [0]:
col_train_generator = new_generator(table_generator, col_mask_generator)
cell_train_generator = new_generator(table_generator, cell_mask_generator)

In [0]:
print(col_train_generator)

In [0]:
tab = table_generator.next()[0]
print(type(tab))
print(len(tab))
print(tab)

In [0]:
tab = col_mask_generator.next()[0]
print(type(tab))
print(len(tab))
print(tab)

In [0]:
# Sprawdzenie transpozycji
arr = np.array([
       [[1, 1, 1], [2, 2, 2]],
       [[3, 3, 3], [4, 4, 4]],
       [[5, 5, 5], [6, 6, 6]]
])
print(arr)
T = keras.layers.Permute((2, 1, 3), input_shape=(3, 2, 3))
print(T(np.array([arr])))
# Działa

In [0]:
# Sprawdzenie reshapea
arr = np.array([
       [[1, 1, 1], [2, 2, 2]],
       [[3, 3, 3], [4, 4, 4]],
       [[5, 5, 5], [6, 6, 6]]
])
print(arr)
T = keras.layers.Permute((2, 1, 3), input_shape=(3, 2, 3))
P = keras.layers.Reshape((2, 9))
w1 = P(T(np.array([arr])))
print(w1)
P2 = keras.layers.Reshape((2, 3, 3))
w2 = P2(w1)
print(w2)
# Działa

In [0]:
col_model = keras.Sequential([
  keras.layers.Permute((2, 1, 3), input_shape=(331, 701, 3)), #Transpozycja obrazka.
  keras.layers.Reshape((701, 993, 1)), #Pozbycie sie kolorow.
  keras.layers.Convolution2D(filters=32, kernel_size=(50, 50), padding='same', input_shape=(701, 993, 1)),
  keras.layers.Convolution2D(filters=16, kernel_size=(25, 25), padding='same', input_shape=(701, 993, 1)),
  keras.layers.Convolution2D(filters=1, kernel_size=(1, 993), input_shape=(701, 993, 1)), #Przejscie w poziomie. 
  keras.layers.Reshape((701, 1)),  #Wywalenie niepotrzebnego wymiaru.
  keras.layers.Convolution1D(filters=8, padding='same', kernel_size=30, activation='sigmoid'),
  keras.layers.Convolution1D(filters=4, padding='same', kernel_size=20, activation='sigmoid'),
  keras.layers.Convolution1D(filters=2, padding='same', kernel_size=10, activation='sigmoid'),
  keras.layers.Convolution1D(filters=1, padding='same', kernel_size=3, activation='hard_sigmoid'), #Dokladna lokalizacja kolumn.

  keras.layers.Reshape([701]),
  keras.layers.RepeatVector(993), #odtworzenie wysokosci
  keras.layers.Reshape((3, 331, 701)),
  keras.layers.Permute((2, 3, 1), input_shape=(3, 331, 701)), #Obrot
])

In [0]:
col_model.compile(loss='mse', optimizer='rmsprop')

In [0]:
his = col_model.fit_generator(
    col_train_generator,
    #use_multiprocessing=True,
    steps_per_epoch=30,
    epochs=10,
    validation_data=col_train_generator,
    validation_steps=10)

In [0]:
tab = table_generator.next()[0]

In [0]:
array_to_img(tab)

In [0]:
prd = col_model.predict(np.array([tab]))

In [0]:
print(prd[0])

In [0]:
array_to_img((prd[0]))